## **М19-ИВТ3**
## **Тотменина Елена**
## **Детектор - вариант 1**
## **Сеть - вариант 5**

### Подготовка: монтирование диска, загрузка и импорт модулей

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from keras.models import Model
from keras.layers import Conv2D, Activation, Input, Add
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.pooling import MaxPooling2D
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

### DeepID модель

In [14]:
def create_deepid_keras_model():
  myInput = Input(shape=(55, 47, 3))
  
  x = Conv2D(20, (4, 4), name='Conv1', activation='relu', input_shape=(55, 47, 3))(myInput)
  x = MaxPooling2D(pool_size=2, strides=2, name='Pool1')(x)
  x = Dropout(rate=0.99, name='D1')(x)
  
  x = Conv2D(40, (3, 3), name='Conv2', activation='relu')(x)
  x = MaxPooling2D(pool_size=2, strides=2, name='Pool2')(x)
  x = Dropout(rate=0.99, name='D2')(x)
  
  x = Conv2D(60, (3, 3), name='Conv3', activation='relu')(x)
  x = MaxPooling2D(pool_size=2, strides=2, name='Pool3')(x)
  x = Dropout(rate=0.99, name='D3')(x)
  
  x1 = Flatten()(x)
  fc11 = Dense(160, name = 'fc11')(x1)
  
  x2 = Conv2D(80, (2, 2), name='Conv4', activation='relu')(x)
  x2 = Flatten()(x2)
  fc12 = Dense(160, name = 'fc12')(x2)
  
  y = Add()([fc11, fc12])
  y = Activation('relu', name = 'deepid')(y)
  
  model = Model(inputs=[myInput], outputs=y)

  return model

### Нахождение лица с помощью классификатора haarcascade

In [15]:
def detectFace(img, arr):
  target_size = (47, 55)
  face_cascade = cv2.CascadeClassifier("/content/drive/My Drive/Colab Notebooks/haarcascade_frontalface_default.xml")
  faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=6, minSize=(20, 20))

  if (len(faces) > 0):
    height, _ = img.shape[:2]
    x, y, w, h = faces[0]
    if y < (height / 2): 
      arr.append(x)
      arr.append(y)
      detected_face = img[int(y):int(y+h), int(x):int(x+w)]
      detected_face = cv2.resize(detected_face, target_size)
      cv2.rectangle(img, (x, y), (x + w, y + h +5), (0, 0, 255), 3)
      img_pixels = image.img_to_array(detected_face)
      img_pixels = np.expand_dims(img_pixels, axis = 0)

      if True:
        img_pixels /= 255 
      else:
        img_pixels /= 127.5
        img_pixels -= 1

      return img_pixels

### Тестирование

In [16]:
video_path = ('/content/drive/My Drive/Colab Notebooks/camera1.mp4')
video = cv2.VideoCapture(video_path) 

model = create_deepid_keras_model()
model.load_weights("/content/drive/My Drive/Colab Notebooks/deepid_keras_weights.h5")

img_path = cv2.imread("/content/drive/My Drive/Colab Notebooks/test2.jpg")
arr = []
img_reference = detectFace(img_path, arr)
source_img = model.predict(img_reference)

while(True):
  grab, frame = video.read()
  if grab == False:
    break

  arr = []
  find_face = detectFace(frame, arr)

  if find_face is not None:
    predict_img = model.predict(find_face)
    cosine = cosine_similarity(source_img, predict_img)
    cv2.putText(frame, 'Similarity:{:.2f}'.format(cosine[0][0]), (arr[0], arr[1]),
    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)

  cv2_imshow(frame)

cv2.destroyAllWindows()